# Lectura y balanceo de datos

## Lectura

In [34]:
#Paquetes
import tensorflow as tf
import numpy as np

#Extracción de datos
lote = 32 #Numero de muestras con el que se trabaja hasta que se actualizan los parametros
data_training = tf.keras.preprocessing.image_dataset_from_directory( #Se necesita TensorFlow version 2.3.0 o superior
    "PetImages", #Ruta de los datos
    label_mode = 'int', #Las imágenes se etiquetan como enteros
    color_mode = 'rgb', #Las imágenes son en color
    batch_size=lote, #Tamaño del lote
    validation_split=0.2, #Proporcion de muestras de validacion
    image_size=(180, 180), #Tamaño en píxeles de la imagen
    subset="training", #Tipo de subset
    seed=1337, #Semilla desde la que empiezo a coger muestras
)
data_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    label_mode = 'int', 
    color_mode = 'rgb',
    batch_size=lote, 
    validation_split=0.2, 
    image_size=(180, 180),
    subset="validation",
    seed=1337,
)

#Número de lotes
print(
    "Número de lotes de entrenamiento: %d"
    % tf.data.experimental.cardinality(data_training)
)
print(
    "Número de lotes de validación: %d" % tf.data.experimental.cardinality(data_validation)
)

In [33]:
#Importamos el paquete para graficar
import matplotlib.pyplot as plt

#Generamos una figura de 10x10
plt.figure(figsize=(10, 10))

for imagen, etiqueta in data_training.take(1):
    #Pintaremos las 9 primeras imágenes
    for i in range(9):
        #Usamos subplot para pintar
        ax = plt.subplot(3, 3, i + 1) #3 filas, 3 columnas y la posicion especificada por i
        plt.imshow(imagen[i].numpy().astype("uint8")) #Mostramos la imagen
        plt.title(int(etiqueta[i])) #Y su titulo es la etiqueta asignada
        plt.axis("off") #Borramos los ejes para mejorar la visualizacion 

# Preparación de las imágenes

In [6]:
#Aumento de datos usando rotaciones 

#Importamos los paquetes necesarios
from tensorflow import keras
from tensorflow.keras import layers

#Usamos una red neuronal de preprocesado para ello
aumentoDatos = keras.Sequential( #Para elaborar una red neuronal por capas llamamos al método sequential
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"), #Capa que ejecuta un giro aleatorio sobre el eje horizontal
        layers.experimental.preprocessing.RandomRotation(0.1), #Giro aleatorio de valor [-2pi*0,2 , 2pi*0,2]
    ]
)

In [32]:
plt.figure(figsize=(10, 10))
for imagen, _ in data_training.take(1): #Cogemos la primera imagen
    for i in range(9):
        nuevaImagen = aumentoDatos(imagen) #Generamos una nueva imagen
        #Y la pintamos como antes
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(nuevaImagen[0].numpy().astype("uint8"))
        plt.axis("off")

In [31]:
nuevaImagen[0]

In [21]:
#Primero guardamos las imagenes aumentadas con la función lambda
data_training_aumentado = data_training.map(lambda x, y: (aumentoDatos(x, training=True), y))

#Después normalizamos
data_training_aumentado = data_training_aumentado.map(lambda x, y:(x/255))

In [30]:
#Mostramos la imagen normalizada
for imagen in data_training_aumentado.take(1): 
    print(imagen[0].numpy())